<a href="https://colab.research.google.com/github/pofeng/pofeng.github.io/blob/master/notebooks/subtitle_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安裝並使用 yt-dlp 下載自動生成字幕

In [ ]:
!pip install yt-dlp

# @markdown ▋ 指定影片網址
url = "https://www.youtube.com/watch?v=DL82mGde6wo" #@param {type:"string"}

!yt-dlp --write-auto-sub --skip-download --sub-lang en --convert-subs srt -o "dl.srt" "{url}"

[youtube] Extracting URL: https://www.youtube.com/watch?v=DL82mGde6wo
[youtube] DL82mGde6wo: Downloading webpage
[youtube] DL82mGde6wo: Downloading tv client config
[youtube] DL82mGde6wo: Downloading player 9fe2e06e-main
[youtube] DL82mGde6wo: Downloading tv player API JSON
[youtube] DL82mGde6wo: Downloading ios player API JSON
[youtube] DL82mGde6wo: Downloading m3u8 information
[info] DL82mGde6wo: Downloading subtitles: en
[info] DL82mGde6wo: Downloading 1 format(s): 616+251
[info] Writing video subtitles to: dl.srt.en.vtt
[download] Destination: dl.srt.en.vtt
[download] 100% of  296.38KiB in 00:00:00 at 5.17MiB/s
[SubtitlesConvertor] Converting subtitles
Deleting original file dl.srt.en.vtt (pass -k to keep)


# 顯示字幕檔 (dl.srt.en.srt)

In [ ]:
!cat dl.srt.en.srt

1
00:00:00,080 --> 00:00:02,070

Metarprompting is turning out to be a

2
00:00:02,070 --> 00:00:02,080
Metarprompting is turning out to be a
 

3
00:00:02,080 --> 00:00:03,830
Metarprompting is turning out to be a
very very powerful tool that everyone's

4
00:00:03,830 --> 00:00:03,840
very very powerful tool that everyone's
 

5
00:00:03,840 --> 00:00:05,590
very very powerful tool that everyone's
using now. It kind of actually feels

6
00:00:05,590 --> 00:00:05,600
using now. It kind of actually feels
 

7
00:00:05,600 --> 00:00:09,030
using now. It kind of actually feels
like coding in you know 1995 like the

8
00:00:09,030 --> 00:00:09,040
like coding in you know 1995 like the
 

9
00:00:09,040 --> 00:00:11,190
like coding in you know 1995 like the
tools are not all the way there. We're

10
00:00:11,190 --> 00:00:11,200
tools are not all the way there. We're
 

11
00:00:11,200 --> 00:00:13,190
tools are not all the way there. We're
you know in this new frontier. But

12
00:00:13,1

# 轉成 HTML

In [ ]:
import re
import urllib.parse

def srt_to_interactive_html(srt_filename, html_filename, video_url):
    """
    將 SRT 檔案轉換為帶有互動式字幕的 HTML 檔案，
    這些字幕可以控制嵌入的 YouTube 播放器。

    Args:
        srt_filename: 輸入 SRT 檔案的路徑。
        html_filename: 輸出 HTML 檔案的路徑。
        video_url: YouTube 影片的完整 URL (例如: "https://www.youtube.com/watch?v=dQw4w9WgXcQ")。
    """

    # --- 從 YouTube URL 中提取影片 ID ---
    parsed_url = urllib.parse.urlparse(video_url)
    video_id = None
    if parsed_url.hostname in ('www.youtube.com', 'youtube.com'):
        # 處理標準 YouTube 連結: https://www.youtube.com/watch?v=VIDEO_ID
        query_params = urllib.parse.parse_qs(parsed_url.query)
        if 'v' in query_params:
            video_id = query_params['v'][0]
    elif parsed_url.hostname in ('youtu.be',):
        # 處理 YouTube 短連結: https://youtu.be/VIDEO_ID
        video_id = parsed_url.path.lstrip('/')

    if not video_id:
        print(f"錯誤：無法從提供的 YouTube URL 中提取影片 ID：{video_url}。請確認 URL 格式是否正確。")
        return

    # --- 時間轉換輔助函數 ---
    def srt_time_to_seconds(time_str):
        """將 SRT 時間格式 (HH:MM:SS,ms) 轉換為秒數 (浮點數)。"""
        h, m, s_ms = time_str.split(':')
        s, ms = s_ms.split(',')
        return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000.0

    def format_seconds_to_hms(seconds):
        """將秒數轉換為 HH:MM:SS 格式的字串。"""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours:02}:{minutes:02}:{secs:02}"

    # --- 寫入 HTML 檔案 ---
    with open(srt_filename, 'r', encoding='utf-8') as srt_file, \
         open(html_filename, 'w', encoding='utf-8') as html_file:

        html_file.write("<!DOCTYPE html>\n<html>\n<head>\n")
        html_file.write("<meta charset=\"UTF-8\">\n")
        html_file.write("<meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">\n") # 響應式設計
        html_file.write("<title>互動式字幕</title>\n")
        html_file.write("<style>\n")
        html_file.write("  /* 基本佈局 */\n")
        html_file.write("  body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin: 0; display: flex; height: 100vh; overflow: hidden; background-color: #f5f5f5; }\n")
        html_file.write("  /* 左側字幕面板 */\n")
        html_file.write("  #left-panel { flex: 1; padding: 20px; overflow-y: auto; background-color: #ffffff; border-right: 1px solid #e0e0e0; box-shadow: 2px 0 5px rgba(0,0,0,0.05); }\n")
        html_file.write("  /* 右側影片面板 */\n")
        html_file.write("  #right-panel { flex: 2; display: flex; justify-content: center; align-items: center; background-color: #000000; position: relative; }\n")
        html_file.write("  /* 影片播放器容器，確保填滿面板 */\n")
        html_file.write("  #player-container { width: 100%; height: 100%; position: absolute; top: 0; left: 0; }\n")

        html_file.write("  /* 字幕行樣式 */\n")
        html_file.write("  .subtitle-line { cursor: pointer; padding: 8px 10px; margin-bottom: 4px; border-radius: 5px; transition: background-color 0.2s ease, transform 0.1s ease; line-height: 1.5; }\n")
        html_file.write("  .subtitle-line:hover { background-color: #e0f7fa; transform: translateY(-1px); }\n")
        html_file.write("  .subtitle-line:active { transform: translateY(0); }\n")
        html_file.write("  /* 高亮顯示當前播放的字幕 */\n")
        html_file.write("  .subtitle-line.active { background-color: #c8e6c9; font-weight: bold; border: 1px solid #81c784; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }\n")
        html_file.write("  /* 時間戳樣式 */\n")
        html_file.write("  .timestamp { font-weight: bold; color: #607d8b; margin-right: 10px; min-width: 70px; display: inline-block; }\n")
        html_file.write("</style>\n")
        html_file.write("</head>\n<body>\n")

        html_file.write("<div id=\"left-panel\">\n")
        html_file.write("  <h2>影片字幕</h2>\n")

        # 用於匹配 SRT 檔案中的時間戳行
        timestamp_regex = re.compile(r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})')

        # 用於暫存 SRT 塊的資料
        current_subtitle_text = []
        current_start_seconds = None
        current_end_seconds = None

        for line in srt_file:
            line = line.strip()

            if line.isdigit(): # 遇到字幕編號行
                # 如果有先前累積的字幕文字，則先將其寫入 HTML
                if current_subtitle_text and current_start_seconds is not None:
                    display_text = " ".join(current_subtitle_text)
                    html_file.write(
                        f"<div class=\"subtitle-line\" data-start-time=\"{current_start_seconds:.3f}\" data-end-time=\"{current_end_seconds:.3f}\">\n"
                        f"  <span class=\"timestamp\">{format_seconds_to_hms(current_start_seconds)}</span>\n"
                        f"  <span class=\"text\">{display_text}</span>\n"
                        f"</div>\n"
                    )
                # 重設暫存變數以準備處理新的字幕塊
                current_subtitle_text = []
                current_start_seconds = None
                current_end_seconds = None
                continue # 跳過字幕編號行

            match = timestamp_regex.match(line)
            if match: # 遇到時間戳行
                # 如果在遇到新的時間戳之前有未寫入的字幕文字，先寫入
                if current_subtitle_text and current_start_seconds is not None:
                    display_text = " ".join(current_subtitle_text)
                    html_file.write(
                        f"<div class=\"subtitle-line\" data-start-time=\"{current_start_seconds:.3f}\" data-end-time=\"{current_end_seconds:.3f}\">\n"
                        f"  <span class=\"timestamp\">{format_seconds_to_hms(current_start_seconds)}</span>\n"
                        f"  <span class=\"text\">{display_text}</span>\n"
                        f"</div>\n"
                    )
                    current_subtitle_text = [] # 清空已處理的文字

                # 解析新的時間戳
                start_time_str = match.group(1)
                end_time_str = match.group(2)
                current_start_seconds = srt_time_to_seconds(start_time_str)
                current_end_seconds = srt_time_to_seconds(end_time_str)
            elif not line: # 遇到空白行 (通常表示字幕塊的結束)
                if current_subtitle_text and current_start_seconds is not None:
                    display_text = " ".join(current_subtitle_text)
                    html_file.write(
                        f"<div class=\"subtitle-line\" data-start-time=\"{current_start_seconds:.3f}\" data-end-time=\"{current_end_seconds:.3f}\">\n"
                        f"  <span class=\"timestamp\">{format_seconds_to_hms(current_start_seconds)}</span>\n"
                        f"  <span class=\"text\">{display_text}</span>\n"
                        f"</div>\n"
                    )
                current_subtitle_text = []
                current_start_seconds = None
                current_end_seconds = None
            elif line: # 遇到字幕文字行
                current_subtitle_text.append(line)

        # 處理檔案結束時可能剩餘的最後一個字幕塊
        if current_subtitle_text and current_start_seconds is not None:
            display_text = " ".join(current_subtitle_text)
            html_file.write(
                f"<div class=\"subtitle-line\" data-start-time=\"{current_start_seconds:.3f}\" data-end-time=\"{current_end_seconds:.3f}\">\n"
                f"  <span class=\"timestamp\">{format_seconds_to_hms(current_start_seconds)}</span>\n"
                f"  <span class=\"text\">{display_text}</span>\n"
                f"</div>\n"
            )

        html_file.write("</div>\n") # 結束 left-panel

        html_file.write("<div id=\"right-panel\">\n")
        html_file.write(f"  <div id=\"player-container\"><div id=\"player\"></div></div>\n")
        html_file.write("</div>\n") # 結束 right-panel

        html_file.write("<script>\n")
        html_file.write("  var player; // YouTube 播放器實例\n")
        html_file.write("  var updateInterval; // 用於高亮字幕的定時器\n")
        html_file.write("  var currentSubtitleIndex = -1; // 當前高亮的字幕索引\n")
        html_file.write("\n")
        html_file.write("  // 當 YouTube IFrame Player API 準備好時呼叫此函數\n")
        html_file.write("  function onYouTubeIframeAPIReady() {\n")
        html_file.write(f"    player = new YT.Player('player', {{\n")
        html_file.write(f"      videoId: '{video_id}',\n")
        html_file.write("      playerVars: { 'autoplay': 0, 'controls': 1, 'rel': 0, 'modestbranding': 1 },\n")
        html_file.write("      events: {\n")
        html_file.write("        'onReady': onPlayerReady,\n")
        html_file.write("        'onStateChange': onPlayerStateChange\n")
        html_file.write("      }\n")
        html_file.write("    });\n")
        html_file.write("  }\n")
        html_file.write("\n")
        html_file.write("  // 當播放器準備好時呼叫此函數\n")
        html_file.write("  function onPlayerReady(event) {\n")
        html_file.write("    var subtitleLines = document.querySelectorAll('.subtitle-line');\n")
        html_file.write("\n")
        html_file.write("    // 為每個字幕行添加點擊事件監聽器\n")
        html_file.write("    subtitleLines.forEach(function(line) {\n")
        html_file.write("      line.addEventListener('click', function() {\n")
        html_file.write("        var startTime = parseFloat(this.getAttribute('data-start-time'));\n")
        html_file.write("        if (player && !isNaN(startTime)) {\n")
        html_file.write("          player.seekTo(startTime, true); // 跳轉到時間點並播放\n")
        html_file.write("          // 立即更新高亮，無需等待定時器\n")
        html_file.write("          updateSubtitleHighlight(startTime);\n")
        html_file.write("        }\n")
        html_file.write("      });\n")
        html_file.write("    });\n")
        html_file.write("  }\n")
        html_file.write("\n")
        html_file.write("  // 根據播放器狀態改變來啟動或停止字幕高亮更新\n")
        html_file.write("  function onPlayerStateChange(event) {\n")
        html_file.write("    if (event.data == YT.PlayerState.PLAYING) {\n")
        html_file.write("      if (!updateInterval) {\n")
        html_file.write("        updateInterval = setInterval(updateSubtitleHighlight, 250); // 每 250ms 檢查一次\n")
        html_file.write("      }\n")
        html_file.write("    } else {\n")
        html_file.write("      clearInterval(updateInterval);\n")
        html_file.write("      updateInterval = null;\n")
        html_file.write("      // 如果暫停或結束，移除高亮\n")
        html_file.write("      if (currentSubtitleIndex !== -1) {\n")
        html_file.write("        var subtitleLines = document.querySelectorAll('.subtitle-line');\n")
        html_file.write("        if (subtitleLines[currentSubtitleIndex]) {\n")
        html_file.write("          subtitleLines[currentSubtitleIndex].classList.remove('active');\n")
        html_file.write("        }\n")
        html_file.write("        currentSubtitleIndex = -1;\n")
        html_file.write("      }\n")
        html_file.write("    }\n")
        html_file.write("  }\n")
        html_file.write("\n")
        html_file.write("  // 更新字幕高亮和捲動位置的函數\n")
        html_file.write("  function updateSubtitleHighlight(manualTime = null) {\n")
        html_file.write("    var currentTime = manualTime !== null ? manualTime : player.getCurrentTime();\n")
        html_file.write("    var subtitleLines = document.querySelectorAll('.subtitle-line');\n")
        html_file.write("\n")
        html_file.write("    var newActiveIndex = -1;\n")
        html_file.write("    for (var i = 0; i < subtitleLines.length; i++) {\n")
        html_file.write("      var line = subtitleLines[i];\n")
        html_file.write("      var startTime = parseFloat(line.getAttribute('data-start-time'));\n")
        html_file.write("      var endTime = parseFloat(line.getAttribute('data-end-time'));\n")
        html_file.write("\n")
        html_file.write("      if (currentTime >= startTime && currentTime < endTime) {\n")
        html_file.write("        newActiveIndex = i;\n")
        html_file.write("        break;\n")
        html_file.write("      }\n")
        html_file.write("    }\n")
        html_file.write("\n")
        html_file.write("    if (newActiveIndex !== currentSubtitleIndex) {\n")
        html_file.write("      // 移除舊的 highlihgt\n")
        html_file.write("      if (currentSubtitleIndex !== -1 && subtitleLines[currentSubtitleIndex]) {\n")
        html_file.write("        subtitleLines[currentSubtitleIndex].classList.remove('active');\n")
        html_file.write("      }\n")
        html_file.write("      // 添加新的 highlight\n")
        html_file.write("      if (newActiveIndex !== -1) {\n")
        html_file.write("        var newLine = subtitleLines[newActiveIndex];\n")
        html_file.write("        newLine.classList.add('active');\n")
        html_file.write("        // 確保高亮字幕在視圖內，平滑捲動\n")
        html_file.write("        newLine.scrollIntoView({ behavior: 'smooth', block: 'center' });\n")
        html_file.write("      }\n")
        html_file.write("      currentSubtitleIndex = newActiveIndex;\n")
        html_file.write("    }\n")
        html_file.write("  }\n")
        html_file.write("\n")
        html_file.write("  // 異步載入 YouTube IFrame Player API 代碼。\n")
        html_file.write("  var tag = document.createElement('script');\n")
        html_file.write("  tag.src = \"https://www.youtube.com/iframe_api\";\n")
        html_file.write("  var firstScriptTag = document.getElementsByTagName('script')[0];\n")
        html_file.write("  firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);\n")
        html_file.write("</script>\n")

        html_file.write("</body>\n</html>\n")

# --- 使用範例 ---
# 請將此處的 YouTube 影片 URL 替換為您的實際影片 URL
youtube_video_url = f"{url}" # 替換為您的 YouTube 影片 URL

# 將 SRT 檔案轉換為互動式 HTML
srt_to_interactive_html("dl.srt.en.srt", "dl.srt.en.html", youtube_video_url)

# 下載 HMTL

In [ ]:
from google.colab import files
files.download('dl.srt.en.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 下載字幕檔 dl.srt.en.txt

In [ ]:
!mv dl.srt.en.srt dl.srt.en.txt

from google.colab import files
files.download('dl.srt.en.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>